In [19]:
import numpy as np
import pandas as pd
import seaborn as sns

df = pd.read_csv('Sample-Superstore - Sample-Superstore.tsv', delimiter = "\t")

In [2]:
for col in df:
    df.rename(columns={col: col.lower().replace(' ', '').replace('-', '')}, inplace=True)

In [5]:
df.head()

,rowid,orderid,orderdate,shipdate,shipmode,customerid,customername,segment,country,city,...,postalcode,region,productid,category,subcategory,productname,sales,quantity,discount,profit
0,1,CA-2016-152156,11/8/2016,11/11/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600,2,0.00,41.9136
1,2,CA-2016-152156,11/8/2016,11/11/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400,3,0.00,219.5820
2,3,CA-2016-138688,6/12/2016,6/16/2016,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,...,90036,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200,2,0.00,6.8714
3,4,US-2015-108966,10/11/2015,10/18/2015,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775,5,0.45,-383.0310
4,5,US-2015-108966,10/11/2015,10/18/2015,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680,2,0.20,2.5164


In [15]:
df_monetary = df.groupby('customerid')['sales'].sum()
df_monetary = df_monetary.reset_index()

In [16]:
df_frequency = df.groupby('customerid')['orderid'].count()
df_frequency = df_frequency.reset_index()
df_frequency.columns = ['customerid', 'order_count']

In [18]:
import datetime as dt

df['orderdate'] = pd.to_datetime(df['orderdate'])

# Calculate the most recent order date
most_recent_date = df['orderdate'].max()

# Subtract the order date from the most recent date
df_recency = df.groupby('customerid')['orderdate'].max().reset_index()
df_recency['recency'] = (most_recent_date - df_recency['orderdate']).dt.days
df_recency.drop('orderdate', axis=1, inplace=True)

In [14]:
df_combined = pd.merge(df_monetary, df_frequency, on='customerid')
df_combined = pd.merge(df_combined, df_recency, on='customerid')
df_combined.head()

,customerid,sales,order_count,recency
0,AA-10315,5563.560,11,184
1,AA-10375,1056.390,15,19
2,AA-10480,1790.512,12,259
3,AA-10645,5086.935,18,55
4,AB-10015,886.156,6,415


In [ ]:
sns.boxplot(x ='order_count', data = df_combined)

<AxesSubplot:xlabel='customerid', ylabel='order_count'>

: 

: 

: 

In [7]:
from sklearn.cluster import KMeans

# Select the relevant features for clustering
features = df[["sales", "quantity", "discount", "profit"]]

# Perform feature scaling if necessary
# ...

# Create and fit the K-means model
kmeans = KMeans(n_clusters=3)  # K is the number of clusters you want to create
kmeans.fit(features)

# Get the cluster labels for each data point
cluster_labels = kmeans.labels_

# Get the centroids of each cluster
centroids = kmeans.cluster_centers_

# Print the cluster labels and centroids
print("Cluster Labels:")
print(cluster_labels)
print("\nCluster Centroids:")
print(centroids)


c:\Users\renev\anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


Cluster Labels:
[0 0 0 ... 0 0 0]

Cluster Centroids:
[[1.35899836e+02 3.68341920e+00 1.57550205e-01 1.38122106e+01]
 [1.06088911e+04 5.46666667e+00 1.33333333e-01 2.57844311e+03]
 [1.80667555e+03 5.89316239e+00 1.29551282e-01 2.48616325e+02]]
